In [118]:
import mlagents
import random
import numpy as np
from mlagents_envs.environment import UnityEnvironment as UE

In [119]:
env = UE(file_name="4444_project",seed=1,side_channels=[])
env.reset()

In [120]:
print(list(env.behavior_specs))
behavior_name = list(env.behavior_specs)[0]
print(behavior_name)

['CarBehavior?team=0']
CarBehavior?team=0


In [121]:
print(type(env.behavior_specs[behavior_name]))
print()
print(env.behavior_specs[behavior_name])
spec = env.behavior_specs[behavior_name]

<class 'mlagents_envs.base_env.BehaviorSpec'>

BehaviorSpec(observation_specs=[ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size8')], action_spec=ActionSpec(continuous_size=1, discrete_branches=()))


In [122]:
print(type(spec.observation_specs))
print()
print(type(tuple(spec.observation_specs[0])))
print(spec.observation_specs[0])

<class 'list'>

<class 'tuple'>
ObservationSpec(shape=(8,), dimension_property=(<DimensionProperty.NONE: 1>,), observation_type=<ObservationType.DEFAULT: 0>, name='VectorSensor_size8')


In [123]:
print(type(spec.action_spec))
print()
print(spec.action_spec)

<class 'mlagents_envs.base_env.ActionSpec'>

Continuous: 1, Discrete: ()


In [124]:
decision_steps, terminal_steps = env.get_steps(behavior_name)

In [125]:
print(decision_steps)
print(list(decision_steps))

[0]


In [126]:
tracked_agent = -1
done = False
episode_rewards = 0

print(type(decision_steps.agent_id))
tracked_agent = decision_steps.agent_id[0]

<class 'numpy.ndarray'>


In [127]:
action = spec.action_spec.random_action(len(decision_steps))
print(action)
print(type(action.discrete))
print(action.discrete)
# action = action.continuous

<class 'numpy.ndarray'>
[]


In [128]:
env.set_actions(behavior_name, action)

In [129]:
for episode in range(30):
    env.reset()
    decision_steps, terminal_steps = env.get_steps(behavior_name)
    tracked_agent = -1 # -1 indicates not yet tracking
    done = False # For the tracked_agent
    episode_rewards = 0 # For the tracked_agent
    while not done:
        # Track the first agent we see if not tracking
        # Note : len(decision_steps) = [number of agents that requested a decision]
        if tracked_agent == -1 and len(decision_steps) >= 1:
            tracked_agent = decision_steps.agent_id[0]
        # Generate an action for all agents
        action = spec.action_spec.random_action(len(decision_steps))
        #action = spec.action_spec.empty_action(len(decision_steps))
        #action.add_continuous(np.array([[-0.5]]))
        # Set the actions
        env.set_actions(behavior_name, action)
        # Move the simulation forward
        env.step()
        # Get the new simulation results
        decision_steps, terminal_steps = env.get_steps(behavior_name)
        print(decision_steps.obs)
        if tracked_agent in decision_steps: # The agent requested a decision
            episode_rewards += decision_steps[tracked_agent].reward
#             print(decision_steps[tracked_agent].reward)
        if tracked_agent in terminal_steps: # The agent terminated its episode
            episode_rewards += terminal_steps[tracked_agent].reward
#             print(terminal_steps[tracked_agent].reward)
            done = True
    print(f"Total rewards for episode {episode} is {episode_rewards}")

[array([[ 2.7046487, -1.1772   , 19.816278 , -1.       ,  8.196485 ,
         8.033186 , 12.105111 , 10.531861 ]], dtype=float32)]
[array([[ 3.7809849, -2.1582   , 19.639353 , -1.       ,  8.253044 ,
         7.931584 , 11.061274 , 11.344244 ]], dtype=float32)]
[array([[ 4.398838 , -3.1391995, 19.51026  , -1.       ,  8.121046 ,
         7.9693165, -1.       , 11.586672 ]], dtype=float32)]
[array([[ 2.9182909 ,  0.09942342, 19.509611  , -1.        ,  8.076248  ,
         7.956549  , 12.189226  , 10.536428  ]], dtype=float32)]
[array([[ 0.4744352 ,  0.04627998, 19.79955   , -1.        ,  8.317835  ,
         7.8573413 , 14.858159  ,  9.2374325 ]], dtype=float32)]
[array([[ 1.7930133 ,  0.12716888, 19.60971   , -1.        ,  8.397482  ,
         7.425818  , 13.165407  , 10.32221   ]], dtype=float32)]
[array([[ 0.8589339 ,  0.15539059, 19.731527  , -1.        ,  8.587206  ,
         7.272814  , 15.084656  ,  9.35771   ]], dtype=float32)]
[array([[-1.0938295 ,  0.10712487, 19.762106  , 20.

KeyboardInterrupt: 

In [ ]:
env.close()